To design a system that categorizes candidates into three categories (Best Fit, Moderate Fit, and No Fit), we need to perform a few key steps. These will involve extracting relevant information from all three sources (Candidate portal data, Resumes, and Job Descriptions) and then using machine learning models to match candidates to job descriptions.

Outline of the Approach:
Data Preprocessing:

Extract structured data from Source 1 (Candidate Portal) and Source 2 (Resumes).

Clean, preprocess, and normalize the data.

Feature Extraction:

Extract relevant features from resumes (Candidate Name, Experience, Domains worked, Languages, etc.).

Convert text data like job descriptions and resumes into numerical vectors (using TF-IDF or embeddings like BERT).

Candidate-Job Matching:

For each candidate, calculate a similarity score with the job description based on their features and the job’s description using cosine similarity, word embeddings, or transformers.

Clustering the Candidates:

Use clustering algorithms like K-Means or hierarchical clustering to categorize candidates into the three categories: Best Fit, Moderate Fit, and No Fit.

Model Training:

If required, train a classification model (using labeled data, if available) to classify candidates directly into these three categories based on features.

Below is a simplified version of the ML pipeline. I'll break down the code into different steps, assuming you're using Python for implementation.

### Step 1: Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

# Sample Candidate Data (Source 1)
df_candidates = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'location_preferences': ['New York', 'San Francisco', 'Austin', 'Seattle', 'Boston'],
    'languages_known': ['Python, Java', 'JavaScript, HTML, CSS', 'Python, Java, SQL', 'Java, C++', 'Python, R'],
    'experience': [5, 3, 7, 10, 2]
})

# Sample Resume Data (Source 2)
df_resumes = pd.DataFrame({
    'candidate_name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'resume_text': [
        "Experienced software engineer with a strong background in Python, Java. Worked on large-scale applications in AI and data analysis.",
        "Frontend developer with experience in building responsive web applications using JavaScript, HTML, and CSS.",
        "Senior software engineer with experience in backend development using Python, Java. Strong experience with databases and data processing.",
        "Lead developer with expertise in Java and C++. Managed a team of engineers in building scalable systems.",
        "Junior data scientist skilled in Python, R, machine learning, and statistical analysis."
    ]
})

# Sample Job Description Data (Source 3)
# df_jobs = pd.DataFrame({
#     'position_title': ['Software Engineer', 'Frontend Developer', 'Senior Software Engineer', 'Lead Developer', 'Data Scientist'],
#     'description': [
#         "We are looking for a software engineer with experience in Python and Java. Should have a good understanding of algorithms and problem-solving.",
#         "Frontend developer needed for web application development. Proficiency in JavaScript, HTML, and CSS is required.",
#         "Senior software engineer required with experience in backend technologies like Python, Java. Knowledge of databases is necessary.",
#         "Lead developer with experience in Java and C++, experience in managing teams and building scalable systems.",
#         "Data scientist role for someone skilled in Python, R, machine learning, and statistical analysis. Experience with data manipulation is a plus."
#     ],
#     'experience_required': [4, 2, 6, 8, 3],
#     'domain': ['Software Engineering', 'Frontend Development', 'Software Engineering', 'Software Engineering', 'Data Science'],
#     'location': ['New York', 'San Francisco', 'Austin', 'Seattle', 'Boston']
# })

df_jobs = pd.DataFrame({
    'position_title': ['Software Engineer'],
    'description': [
        "We are looking for a software engineer with experience in Python and Java. Should have a good understanding of algorithms and problem-solving."
        
    ],
    'experience_required': [4],
    'domain': ['Software Engineering'],
    'location': ['New York']
})


### Step 2: Feature Extraction (TF-IDF for Text)

In [2]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# Preprocess Candidate Data
def preprocess_candidate_data(df):
    df['name'] = df['name'].str.lower()
    df['location_preferences'] = df['location_preferences'].str.lower()
    df['languages_known'] = df['languages_known'].str.lower()
    return df

# Preprocess Resume Data
def preprocess_resume_data(df):
    df['resume_text'] = df['resume_text'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))
    return df

# Preprocess Job Description Data
def preprocess_job_description_data(df):
    df['description'] = df['description'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))
    return df

# Apply preprocessing
df_candidates = preprocess_candidate_data(df_candidates)
df_resumes = preprocess_resume_data(df_resumes)
df_jobs = preprocess_job_description_data(df_jobs)

# TF-IDF Vectorizer for Resume Text and Job Description Text
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Combine candidate's resume text and job descriptions for similarity calculation
def compute_tfidf_matrix(resume_texts, job_descriptions):
    resume_tfidf = vectorizer.fit_transform(resume_texts)
    job_desc_tfidf = vectorizer.transform(job_descriptions)
    return resume_tfidf, job_desc_tfidf

# Compute the TF-IDF matrix for resumes and job descriptions
resume_tfidf, job_desc_tfidf = compute_tfidf_matrix(df_resumes['resume_text'], df_jobs['description'])


### Step 3: Calculate Cosine Similarity

In [3]:
# Cosine Similarity Calculation between Candidates' Resumes and Job Descriptions
def calculate_cosine_similarity(resume_tfidf, job_desc_tfidf):
    similarity_matrix = cosine_similarity(resume_tfidf, job_desc_tfidf)
    return similarity_matrix

similarity_matrix = calculate_cosine_similarity(resume_tfidf, job_desc_tfidf)

# Check the similarity matrix
print("Cosine Similarity Matrix:\n", similarity_matrix)


Cosine Similarity Matrix:
 [[0.42039137]
 [0.12674805]
 [0.61010361]
 [0.08981684]
 [0.09670659]]


### Step 4: Candidate Fit Categorization Using Clustering
We will use a clustering algorithm like K-Means to categorize candidates into 3 clusters: Best Fit, Moderate Fit, No Fit.

In [4]:
# Let's assume we're clustering based on similarity scores
def cluster_candidates(similarity_matrix, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    # Reshape similarity matrix to fit k-means input shape (candidate-job similarity)
    kmeans.fit(similarity_matrix)
    return kmeans.labels_

# Cluster candidates into Best Fit, Moderate Fit, No Fit
candidate_clusters = cluster_candidates(similarity_matrix)

# Add the clusters to candidate dataframe
df_candidates['fit_category'] = candidate_clusters

# Map cluster labels to human-readable categories
def map_cluster_to_fit_category(cluster_labels):
    fit_categories = {0: 'Best Fit', 1: 'Moderate Fit', 2: 'No Fit'}
    return [fit_categories[label] for label in cluster_labels]

df_candidates['fit_category'] = map_cluster_to_fit_category(df_candidates['fit_category'])

# Show the final candidates with their fit category
print("Candidates with Fit Categories:\n", df_candidates)


Candidates with Fit Categories:
       name location_preferences        languages_known  experience  \
0    alice             new york           python, java           5   
1      bob        san francisco  javascript, html, css           3   
2  charlie               austin      python, java, sql           7   
3    david              seattle              java, c++          10   
4      eve               boston              python, r           2   

   fit_category  
0        No Fit  
1      Best Fit  
2  Moderate Fit  
3      Best Fit  
4      Best Fit  


# Alternative Approach: Embeddings with Transformers (e.g., BERT)

### Step 1: Install Libraries

In [ ]:
pip install transformers torch scikit-learn
pip install huggingface_hub[hf_xet]
pip install hf_xet

### Step 2: Load BERT Model
We’ll load a pre-trained BERT model from Hugging Face and use it to generate embeddings for the resumes and job descriptions.

In [7]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

# Load the pre-trained BERT tokenizer and model from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Check if GPU is available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### Step 3: Encode Text Using BERT
We'll create a function that converts text into BERT embeddings. BERT returns embeddings for every token in the input, but we'll average these token embeddings to get a single vector representing the entire text.

In [13]:
# Function to encode text using BERT and get the embeddings
def get_bert_embedding(text):
    # Tokenize the text and convert it to BERT's input format
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
    # Pass the tokenized text through BERT
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the last hidden state (token embeddings)
    embeddings = outputs.last_hidden_state
    
    # Take the mean of the token embeddings to get a single vector for the entire text
    pooled_embedding = embeddings.mean(dim=1).squeeze()
    
    return pooled_embedding.cpu().numpy()



### Step 4: Calculate Cosine Similarity
We’ll use the cosine similarity to calculate how similar each candidate’s resume is to the job description.

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate cosine similarity between two vectors
def calculate_cosine_similarity(embedding_1, embedding_2):
    similarity = cosine_similarity([embedding_1], [embedding_2])
    return similarity[0][0]



In [15]:
# Function to check for domain matching using regex
def is_domain_matching(resume_text, domain_keywords):
    """
    Check if any of the domain-related keywords match in the resume using regex.
    """
    resume_text = resume_text.lower()
    for keyword in domain_keywords:
        # Check for a match of any domain-related keyword (case insensitive)
        if re.search(r'\b' + re.escape(keyword) + r'\b', resume_text):
            return True
    return False

# Function to evaluate candidates for each job description
def evaluate_candidates_for_jobs(df_candidates, df_jobs, threshold_best=0.7, threshold_moderate=0.5):
    job_candidates = {}

    # Define the domain-related keywords for matching (can be extended based on your use case)
    domain_keywords_map = {
        'Software Engineering': ['software engineer', 'software engineering', 'software developer', 'developer', 'programmer'],
        'Frontend Development': ['frontend developer', 'web developer', 'javascript', 'html', 'css'],
        'Data Science': ['data scientist', 'machine learning', 'statistical analysis', 'data analysis'],
        'Backend Development': ['backend developer', 'backend engineer', 'server-side', 'database'],
        # Add more as needed...
    }

    # Process each job description
    for _, job_row in df_jobs.iterrows():
        job_desc_embedding = get_bert_embedding(job_row['description'])
        
        # List to store candidates for this job description
        best_fit_candidates = []
        moderate_fit_candidates = []

        print(f"\nEvaluating job: {job_row['position_title']}")

        for _, resume_row in df_candidates.iterrows():
            resume_embedding = get_bert_embedding(resume_row['resume_text'])
            similarity_score = calculate_cosine_similarity(resume_embedding, job_desc_embedding)

            # Check for domain match using domain keywords map
            domain_keywords = domain_keywords_map.get(job_row['domain'], [])
            if not is_domain_matching(resume_row['resume_text'], domain_keywords):
                continue  # Exclude candidates who don't have experience in the job's domain

            # Print cosine similarity scores for debugging
            print(f"Similarity between '{resume_row['name']}' and job '{job_row['position_title']}': {similarity_score:.4f}")

            # Classify based on similarity threshold
            if similarity_score > threshold_best:
                best_fit_candidates.append(resume_row['name'])
            elif similarity_score > threshold_moderate:
                moderate_fit_candidates.append(resume_row['name'])

        # Combine best fit and moderate fit candidates into a final list
        candidates_for_job = best_fit_candidates + moderate_fit_candidates

        # Store the job and corresponding candidates' names (comma separated)
        job_candidates[job_row['position_title']] = ', '.join(candidates_for_job) if candidates_for_job else "No candidates"

    return job_candidates

This will output the similarity score between the resume and the job description.

### Step 5: Classify Candidates Based on Similarity
Now, let’s extend the process to classify multiple candidates based on the similarity score. We can use a threshold-based classification as before.

In [16]:
# Sample Candidate Data (Source 1)
df_candidates = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'location_preferences': ['New York', 'San Francisco', 'Austin', 'Seattle', 'Boston'],
    'languages_known': ['Python, Java', 'JavaScript, HTML, CSS', 'Python, Java, SQL', 'Java, C++', 'Python, R'],
    'experience': [5, 3, 7, 10, 2]
})

# Add the resume text column
df_candidates['resume_text'] = [
    "Experienced software engineer with a strong background in Python, Java. Worked on large-scale applications in AI and data analysis.",
    "Frontend developer with experience in building responsive web applications using JavaScript, HTML, and CSS.",
    "Senior software engineer with experience in backend development using Python, Java. Strong experience with databases and data processing.",
    "Lead developer with expertise in Java and C++. Managed a team of engineers in building scalable systems.",
    "Junior data scientist skilled in Python, R, machine learning, and statistical analysis."
]

# Sample Job Description Data (Source 3)
df_jobs = pd.DataFrame({
    'position_title': ['Software Engineer', 'Frontend Developer', 'Senior Software Engineer', 'Lead Developer', 'Data Scientist'],
    'description': [
        "We are looking for a software engineer with experience in Python and Java. Should have a good understanding of algorithms and problem-solving.",
        "Frontend developer needed for web application development. Proficiency in JavaScript, HTML, and CSS is required.",
        "Senior software engineer required with experience in backend technologies like Python, Java. Knowledge of databases is necessary.",
        "Lead developer with experience in Java and C++, experience in managing teams and building scalable systems.",
        "Data scientist role for someone skilled in Python, R, machine learning, and statistical analysis. Experience with data manipulation is a plus."
    ],
    'experience_required': [4, 2, 6, 8, 3],
    'domain': ['Software Engineering', 'Frontend Development', 'Software Engineering', 'Software Engineering', 'Data Science'],
    'location': ['New York', 'San Francisco', 'Austin', 'Seattle', 'Boston']
})

# Final Results:
# Software Engineer: Alice, Bob, Charlie, David
# Frontend Developer: Bob
# Senior Software Engineer: Alice, Bob, Charlie, David
# Lead Developer: Alice, Bob, Charlie, David
# Data Scientist: Alice, Eve

# Run the evaluation
job_candidates = evaluate_candidates_for_jobs(df_candidates, df_jobs)

# Print the final output (Job Description and Candidates)
print("\nFinal Results:")
for job, candidates in job_candidates.items():
    print(f"{job}: {candidates}")


Evaluating job: Software Engineer
Similarity between 'Alice' and job 'Software Engineer': 0.8755
Similarity between 'Bob' and job 'Software Engineer': 0.7730
Similarity between 'Charlie' and job 'Software Engineer': 0.8817
Similarity between 'David' and job 'Software Engineer': 0.8266

Evaluating job: Frontend Developer
Similarity between 'Bob' and job 'Frontend Developer': 0.9222

Evaluating job: Senior Software Engineer
Similarity between 'Alice' and job 'Senior Software Engineer': 0.8767
Similarity between 'Bob' and job 'Senior Software Engineer': 0.8111
Similarity between 'Charlie' and job 'Senior Software Engineer': 0.9162
Similarity between 'David' and job 'Senior Software Engineer': 0.8558

Evaluating job: Lead Developer
Similarity between 'Alice' and job 'Lead Developer': 0.9081
Similarity between 'Bob' and job 'Lead Developer': 0.8986
Similarity between 'Charlie' and job 'Lead Developer': 0.9258
Similarity between 'David' and job 'Lead Developer': 0.9584

Evaluating job: Data